In [ ]:
# maybe help mpi setup
import mpi4py.rc
mpi4py.rc.threaded = False
mpi4py.rc.thread_level = "single"

# import all python modules
from mpi4py import MPI
import underworld as uw
import underworld.function as fn
import glucifer
import os
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Image
from IPython.display import clear_output
from ipywidgets import Button, HBox, VBox

from tqdm import tqdm

import unsupported.geodynamics.scaling as sca
from unsupported.geodynamics.scaling import nonDimensionalize as nd

comm = MPI.COMM_WORLD

# set output path
outputPath='model_b'
outputPath = os.path.abspath(outputPath)+'/'
# build output path on root proc
if uw.rank() == 0:
    if not os.path.exists(outputPath):
        print "Making outputPath, ", outputPath
        os.makedirs(outputPath)


In [ ]:
def checkpoint( mesh, fieldDict, swarm, swarmDict, index, outputDir='./',
                    meshName='mesh', swarmName='swarm',
                    enable_xdmf=True, with_deform_mesh=False):

        prefix = outputDir
        # Check the prefix is valid
        if not os.path.exists(prefix) and uw.rank()==0:
            raise RuntimeError("Error checkpointing, can't find {}".format(prefix))
        if not isinstance(index, int):
            raise TypeError("'index' is not of type int")
        ii = str(index).zfill(4)

        # only is there's a mesh then check for fields to save
        if mesh is not None:

            # Error check the mesh and fields
            if not isinstance(mesh, uw.mesh.FeMesh):
                raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
            if not isinstance(fieldDict, dict):
                raise TypeError("'fieldDict' is not of type dict")
            for key, value in fieldDict.iteritems():
                if not isinstance( value, uw.mesh.MeshVariable ):
                    raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")

            # Save the mesh each call of checkpoint() if with_deform_mesh is enabled
            if with_deform_mesh is True:
                mh = mesh.save(prefix+meshName+".h5")
            else:
                # see if we have already saved the mesh. It only needs to be saved once
                if not hasattr( checkpoint, 'mH' ):
                    checkpoint.mH = mesh.save(prefix+meshName+".h5")
                mh = checkpoint.mH

            # save xdmf files
            for key,value in fieldDict.iteritems():
                filename = prefix+key+'-'+ii
                handle = value.save(filename+'.h5', mh)
                if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName)

        # is there a swarm
        if swarm is not None:

            # Error check the swarms
            if not isinstance(swarm, uw.swarm.Swarm):
                raise TypeError("'swarm' is not of type uw.swarm.Swarm")
            if not isinstance(swarmDict, dict):
                raise TypeError("'swarmDict' is not of type dict")
            for key, value in swarmDict.iteritems():
                if not isinstance( value, uw.swarm.SwarmVariable ):
                    raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")

            # save xdmf files
            sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
            for key,value in swarmDict.iteritems():
                filename = prefix+key+'-'+ii
                handle = value.save(filename+'.h5')
                if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName)


In [ ]:
# reference units setup
u = sca.UnitRegistry

# desired scaling
KL_meters = 1 * u.kilometer
K_viscosity = (1e18 * u.pascal * u.second)
K_density = (1e3 * u.kilogram / (u.meter)**3 )

KM_kilograms = K_density * KL_meters**3
KT_seconds = KM_kilograms / ( KL_meters * K_viscosity )
K_substance = 1. * u.mole
Kt_degrees = 1. * u.kelvin

sca.scaling["[length]"] = KL_meters.to_base_units()
sca.scaling["[temperature]"] = Kt_degrees.to_base_units()
sca.scaling["[time]"] = KT_seconds.to_base_units()
sca.scaling["[mass]"] = KM_kilograms.to_base_units()

In [ ]:
fs = {'description_width': 'initial'} # full style
# model parameters
_Lx = widgets.BoundedFloatText(value=1000,min=1,max=1000.0,
                               description='Model width [km]', style=fs,disabled=False )

_Ly = widgets.BoundedFloatText(value=660,min=1,max=1000.0,
                               description='Model height [km]',style=fs, disabled=False )

_air_depth   = widgets.BoundedFloatText(value=40,min=1,max=1000.0,
                                        description='air depth [km]',style=fs, disabled=False )

_plate_depth = widgets.BoundedFloatText(value=80,min=1,max=1000.0,
                                        description='plat depth [km]',style=fs, disabled=False )

_slab_length = widgets.BoundedFloatText(value=250,min=1,max=1000.0,
                                        description='slab length [km]',style=fs, disabled=False )
_slab_width  = widgets.BoundedFloatText(value=80,min=1,max=1000.0,
                                        description='slab width [km]',style=fs, disabled=False )

_max_time = widgets.BoundedFloatText(value=22.56,min=1,max=50,
                                     description='Model time [Myr]',style=fs, disabled=False )

# The following MUST be in SI units
_min_viscosity = 1e18
_max_viscosity = 1e25

_gravity = 9.81

_mantle_density     = 0
_mantle_viscosity   = 1e21 # for case 1 only

_slab_density       = 150
slab_exponent      = 4.
mu                  = 4.75e11 # issue here


In [ ]:
outputbox = widgets.Text(
    value='output/slabdetachment',
    description='output path:',
    disabled=False
)

w_button = widgets.Button(
    description='Do science',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

In [ ]:
def display_options():
    display(Image('../images/SlabDetachment.png', width=500))
    row1 = HBox([_Lx, _Ly])
    row2 = HBox([_air_depth, _plate_depth])
    row3 = HBox([_slab_length, _slab_width])
    display(VBox([row1, row2, row3]))
    display(_max_time)
    display(outputbox)
    display(w_button)

In [ ]:
def make_model(b):
#     clear_output()
#     display_options()
    global outputPath 
    outputPath = str(outputbox.value)
    import os
    outputPath = os.path.abspath(outputPath)+'/'
    # build output path on root proc
    if uw.rank() == 0:
        if not os.path.exists(outputPath):
            os.makedirs(outputPath)
    
    Lx = nd( _Lx.value * u.kilometer )
    Ly = nd( _Ly.value * u.kilometer )

    air_depth   = nd(  _air_depth.value * u.kilometer )
    plate_depth = nd(  _plate_depth.value * u.kilometer)
    slab_length = nd( _slab_length.value * u.kilometer ) 
    slab_width  = nd( _slab_width.value * u.kilometer )

    max_time = nd(_max_time.value *u.megayear )
    min_viscosity = nd(_min_viscosity * u.Pa*u.sec)
    max_viscosity = nd(_max_viscosity* u.Pa*u.sec)

    gravity = nd(_gravity  * u.meter / u.second**2)

    mantle_density     = nd(_mantle_density * u.kilogram / u.meter**3)
    mantle_viscosity   = nd(_mantle_viscosity * u.Pa*u.sec) # for case 1 only

    slab_density       = nd(_slab_density * u.kilogram / u.meter**3)
    _mu                 = nd(mu * u.Pa*u.sec**(1/slab_exponent) )

    resFactor = 2
    resX = resFactor * 100
    resY = resFactor * 66

    elRes = (resX, resY)
    minCoord = [0.,0.]
    maxCoord = [Lx, Ly]
    elType   = "Q1/dq0"
    
    # FEM mesh and variable setup
    mesh = uw.mesh.FeMesh_Cartesian(maxCoord   = maxCoord, 
                                    minCoord   = minCoord,
                                    elementRes = elRes, 
                                    elementType = elType)
    print("Running with {} x {} elements".format(resX, resY))
    vField = mesh.add_variable(2)
    pField = mesh.subMesh.add_variable(1)
    etaField = mesh.add_variable(1)

    # derivative functions of FE variables
    vdotv = fn.math.dot( vField, vField )
    srFn = fn.tensor.symmetric( vField.fn_gradient )
    sr_2InvFn = fn.tensor.second_invariant(srFn) # 2nd invariant of strain rate 

    # lagrangian swarm setup - stores material and viscosity
    swarm        = uw.swarm.Swarm(mesh)
    advector     = uw.systems.SwarmAdvector(vField, swarm, order=2)
    materialVar  = swarm.add_variable(count=1, dataType='int')
    viscosityVar = swarm.add_variable(count=1, dataType='double')

    layout = uw.swarm.layouts.PerCellSpaceFillerLayout(swarm, particlesPerCell=20)
    swarm.populate_using_layout(layout)

    population_control = uw.swarm.PopulationControl(swarm, 
                                                    aggressive=False,splitThreshold=0.15, maxDeletions=2,maxSplits=10,
                                                    particlesPerCell=20)
    # No-slip side walls, free-slip top and bottom
    iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
    jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
    bottomWall = mesh.specialSets["MinJ_VertexSet"]
    allWalls = iWalls + jWalls
    vField.data[...] = 0.0

    # dirichlet conditions for model
    stokes_dBC = uw.conditions.DirichletCondition( variable        = vField, 
                                                   indexSetsPerDof = (iWalls, allWalls) )
    
    # build plate and slab geometries
    p_h = mesh.maxCoord[1] - plate_depth
    array = np.array([(mesh.minCoord[0], p_h), (mesh.maxCoord[0], p_h),
                      (mesh.maxCoord[0], mesh.maxCoord[1]), (mesh.minCoord[0], mesh.maxCoord[1])])
    slab2_polygon = uw.function.shape.Polygon(vertices=array)

    slab_x0 = mesh.maxCoord[0]/2 - slab_width/2
    slab_x1 = slab_x0 + slab_width
    s_h = p_h - slab_length
    array = np.array([(slab_x0, s_h), (slab_x1, s_h),
                      (slab_x1, Ly ), (slab_x0, Ly )])
    slab_polygon = uw.function.shape.Polygon(vertices=array)

    # assign geometries to swarm via polygon class
    materialVar.data[:] = slab2_polygon.evaluate(swarm) + slab_polygon.evaluate(swarm)
    
    # create analysis swarms

    # Ds - slab tip depth single tracking particle
    ts_ds      = uw.swarm.Swarm(mesh)
    ts_ds_ad   = uw.systems.SwarmAdvector(vField, ts_ds, order=2)
    depthVar   = ts_ds.add_variable(count=1, dataType='double')

    p_coords = np.ndarray(shape=(1, 2))
    p_coords[:,0] = Lx/2.
    p_coords[:,1] = slab_length

    ts_ds.add_particles_with_coordinates(p_coords)
    fn_ds = fn.misc.constant(1.0) - fn.coord()[1]/Ly
    depthVar.data[:] = fn_ds.evaluate(ts_ds)

    # width tracers
    ts_ws      = uw.swarm.Swarm(mesh)
    ts_ws_ad   = uw.systems.SwarmAdvector(vField, ts_ws, order=2)
    dummVar    = ts_ws.add_variable(count=1, dataType='int') # dummy var required to save

    p_coords = np.ndarray(shape=(100, 2))
    p_coords[0:50,0] = slab_x0
    p_coords[0:50,1] = np.linspace(s_h, p_h, num=50)
    p_coords[50:100,0] = slab_x1
    p_coords[50:100,1] = np.linspace(s_h, p_h, num=50)

    ignore=ts_ws.add_particles_with_coordinates(p_coords)
    
    # assign Fns for phyical properties

    # materialVar => (0,1) = (mantle,slab)
    # Density
    densityFn = uw.function.branching.map( fn_key=materialVar, 
                                        mapping={ 1: slab_density, 0: mantle_density } )
    # Force
    forceFn = gravity*densityFn * (0.0,-1.)

    # Viscosity
    dummy_sr_2Inv = fn.misc.constant(nd(1.0e-16 / u.second))

    # strain rate and power-law viscosity
    sr      = fn.misc.max(sr_2InvFn, dummy_sr_2Inv)
    eta_raw = _mu*fn.math.pow(sr, (1./slab_exponent - 1.))

    # viscous limiting
    maxBound       = fn.misc.min(eta_raw,  max_viscosity)
    finalViscosity = fn.misc.max(maxBound, min_viscosity)

    viscosityFn = uw.function.branching.map( fn_key=materialVar, 
                                        mapping={ 1: finalViscosity, 0: mantle_viscosity } )
    
    # setup solver
    stokes = uw.systems.Stokes( velocityField = vField, 
                                pressureField = pField,
                                conditions    = [stokes_dBC],
                                fn_viscosity  = viscosityFn, 
                                fn_bodyforce  = forceFn )
    solver = uw.systems.Solver( stokes )
    if uw.nProcs()==1:
        solver.set_inner_method("lu")
        
    # setup vis
    gldb = glucifer.Store(filename=outputPath+'/glucifer.gldb')
    figVel = glucifer.Figure(store=gldb)
    figVel.VectorArrows(mesh, vField, name="velocity vector")
    figVel.Surface(mesh, sr_2InvFn, logScale=True, name="StrainRate 2ndInv")

    figEta = glucifer.Figure(store=gldb)
    figEta.Points(swarm, viscosityFn, logScale=True, fn_size=2.0, name="Viscosity")

    figEps = glucifer.Figure(store=gldb)
    figEps.Points(swarm, sr_2InvFn, logScale=True, fn_size=2.0, name="StrainRate 2ndInv")

    figMat = glucifer.Figure(store=gldb)
    figMat.Points(swarm, densityFn, name="Density")
    figMat.show()
        
    # Get solution
    solver.solve(nonLinearIterate=True,nonLinearTolerance=1e-2)
    projection = uw.utils.MeshVariable_Projection(etaField, stokes.fn_viscosity)
    projection.solve()
    
    fieldDict = {'velocity':vField, 'viscosityField':etaField}
#     checkpoint( mesh, fieldDict, None, None, 0, outputDir=outputPath) # for initial analysis

    # redefine
    swarmDict = {'viscosity':viscosityVar}
    fieldDict = {'velocity':vField}
    
    # simulation parameters
    steps         = 0
    finalTimestep = 5
    curTime       = 0
    cp_i          = 0

    # Wall time timers
    timer_0, timer_1 = 0,0

    if uw.rank() == 0:
        outfile = open(outputPath+'buildMount.txt', 'w+')
        string = "steps, timestep, time, vrms, slab_tip_depth, Wtime"
#         print(string)
        outfile.write( string+"\n")
        
        pbar = tqdm(total=_max_time.value) # progress bar in SI units

    # initialise loop
    dt = -1
    # h1 = mswarm.particleCoordinates.data[:,1].copy()

    timer_0 = MPI.Wtime()
#     while steps<finalTimestep:
    while (max_time-curTime) > (1e-3*max_time):

        depthVar.data[:] = fn_ds.evaluate(ts_ds)

        # Get solution
        solver.solve(nonLinearIterate=True,nonLinearTolerance=1e-2)

        viscosityVar.data[:] = stokes.fn_viscosity.evaluate(swarm)

        if steps % 5 == 0:
#             checkpoint( mesh, fieldDict, swarm, swarmDict, cp_i, outputDir=outputPath)
            cp_i += 1 # due to paraview 5.4 oddity
        ts_ws.save(outputPath+'/width_swarm.'+str(steps).zfill(4)+'.h5')

        # calculate metrics
        v2int = mesh.integrate(fn=vdotv)[0]
        vol   = mesh.integrate(fn=1.0)[0]

        s_d = 0
        if ts_ds.particleLocalCount>0: 
            s_d = depthVar.data[0][0]
        s_d = comm.allreduce(s_d)

        # get time step first time around
        if dt < 0:
            dt = advector.get_max_dt()
    #     h0 = h1.copy() # NOTE the copy()

        curTime += dt
        real_dt = sca.Dimensionalize(dt, u.megayear).magnitude
        pbar.update(real_dt)
        timer_1 = MPI.Wtime()
        string = "{}, {:.5e}, {:.5e}, {:.5e}, {:.5e}, {:.5e}".format(steps,
                                     sca.Dimensionalize(dt, u.year),
                                     sca.Dimensionalize(curTime, u.year),
                                     sca.Dimensionalize(np.sqrt(v2int/vol), u.cm/u.year), 
                                     s_d,
                                     (timer_1-timer_0) )
        timer_0 = timer_1
#         print(string)
        if uw.rank() == 0:
            outfile.write(string+"\n")

        figEta.save(outputPath+"eta-"+str(steps)+".png")
        figEps.save(outputPath+"eps-"+str(steps)+".png")

        # Advect particles   
        advector.integrate(dt)
        ts_ws_ad.integrate(dt)
        ts_ds_ad.integrate(dt)
        # population control
        population_control.repopulate()

    #     # update peak heigh
    #     h1 = mswarm.particleCoordinates.data[:,1]

    #     diffH = h1-h0

        steps += 1
        gldb.step = steps

    if uw.rank()==0:
        outfile.close()
        pbar.close()

In [ ]:
w_button.on_click(make_model)

In [ ]:
display_options()

In [ ]:
def vizualise(path=None):
    # if no path provided use the global value from the widget
    if path == None:
        global outputPath
        path = outputPath
    import os
    if not os.path.exists(path):
        print("Can't find ", path)
        return

    vis = glucifer.lavavu.Viewer(database=path+'/glucifer.gldb')
    vis.control.Panel()

    vis.control.TimeStepper()
    vis.control.ObjectList()
    vis.control.show()